In [28]:
import os
import openai
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue

# Import aliased to avoid unused import warning until used
from langsmith import Client 

In [3]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [4]:
all_points = qdrant_client.scroll(
   collection_name="Amazon-items-collection-00",
   limit=100,
   offset=None,
   with_payload=True,
   with_vectors=False
)

In [5]:
all_points

([Record(id=0, payload={'description': 'YKZ 3 in 1 Magnetic C to C Cable 4 Pack, Magnet Type C to Type C Charging Cable, [3.3FT+6.0FT] PD 60W Fast Charger Nylon Braided Cord for Type C/MicroUSB and i-Product【3 in 1 USB C to USB C cable】- This cable has 3 different magnet connectors, suitable for Type-c, Micro-USB devices and i-Product. (NOTE: This is the USB-C output cable, you may need a USB C wall charger to charge your device.) 【PD 60W Fast Charge】- YKZ type C to type C Cable supports Power Delivery fast charge up to 20V/3A (max), 15V/3A, 12V/3A, 9V/3A, 5V/3A at 60W with USB-C PD adapters such as PD 18W/30W/45W/60W. (Note: Charging speeds also depends on your device and charger. ) 【Magnetic Charging Cable】- Magnetic charging cable has a strong magnet that can attract the phone without falling off. You can easily connect with one hand while driving or doing other things. Besides, Magnetic tips can be used as a dust plug that reducing abrasion during inserting or plugging out the cabl

In [8]:
all_points[0][0].payload

{'description': 'YKZ 3 in 1 Magnetic C to C Cable 4 Pack, Magnet Type C to Type C Charging Cable, [3.3FT+6.0FT] PD 60W Fast Charger Nylon Braided Cord for Type C/MicroUSB and i-Product【3 in 1 USB C to USB C cable】- This cable has 3 different magnet connectors, suitable for Type-c, Micro-USB devices and i-Product. (NOTE: This is the USB-C output cable, you may need a USB C wall charger to charge your device.) 【PD 60W Fast Charge】- YKZ type C to type C Cable supports Power Delivery fast charge up to 20V/3A (max), 15V/3A, 12V/3A, 9V/3A, 5V/3A at 60W with USB-C PD adapters such as PD 18W/30W/45W/60W. (Note: Charging speeds also depends on your device and charger. ) 【Magnetic Charging Cable】- Magnetic charging cable has a strong magnet that can attract the phone without falling off. You can easily connect with one hand while driving or doing other things. Besides, Magnetic tips can be used as a dust plug that reducing abrasion during inserting or plugging out the cable, greatly prolong the 

In [10]:
all_context = [{"id":data.payload["parent_asin"], "text":data.payload["description"]}for data in all_points[0]]

In [11]:
all_context

[{'id': 'B09NKXF9YQ',
  'text': 'YKZ 3 in 1 Magnetic C to C Cable 4 Pack, Magnet Type C to Type C Charging Cable, [3.3FT+6.0FT] PD 60W Fast Charger Nylon Braided Cord for Type C/MicroUSB and i-Product【3 in 1 USB C to USB C cable】- This cable has 3 different magnet connectors, suitable for Type-c, Micro-USB devices and i-Product. (NOTE: This is the USB-C output cable, you may need a USB C wall charger to charge your device.) 【PD 60W Fast Charge】- YKZ type C to type C Cable supports Power Delivery fast charge up to 20V/3A (max), 15V/3A, 12V/3A, 9V/3A, 5V/3A at 60W with USB-C PD adapters such as PD 18W/30W/45W/60W. (Note: Charging speeds also depends on your device and charger. ) 【Magnetic Charging Cable】- Magnetic charging cable has a strong magnet that can attract the phone without falling off. You can easily connect with one hand while driving or doing other things. Besides, Magnetic tips can be used as a dust plug that reducing abrasion during inserting or plugging out the cable, grea

### Render a prompt to generate synthetic eval reference dataset 

In [ ]:
import json

output_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Suggested question.",
            },
            "chunk_ids": {
                "type": "array",
                "items": {
                    "type": "string",
                    "description": "ID of the chunk that could be used to answer the question.",
                },
            },
            "answer_example": {
                "type": "string",
                "description": "Suggested answer grounded in the context.",
            },
            "reasoning": {
                "type": "string",
                "description": "Reasoning why the question could be answered with the chunks.",
            },
        },
    },
}


SYSTEM_PROMPT = f"""
I am building a RAG application. I have a collection of 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of the products we have available.
I will provide all of the available products to you with IDs of each chunk.
I want you to come up with 30 questions to which the answers could be grounded in the chunk context.
As an output I need you to provide me the list of questions and the IDs of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Construct 10 that could use multiple chunks in the answer.
Construct 15 questions that could use single chunk in the answer.
Also, include 5 questions that can't be answered with the available chunks.

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema, indent=2)}
</OUTPUT JSON SCHEMA>

I need to be able to parse the json output.
"""

USER_PROMPT = f"""
Here is the list of chunks, each list element is a dictionary with id and text:
{all_context}
"""

In [14]:
print(USER_PROMPT)



Here is the list of chunks, each list element is a dictionary with id and text:
[{'id': 'B09NKXF9YQ', 'text': 'YKZ 3 in 1 Magnetic C to C Cable 4 Pack, Magnet Type C to Type C Charging Cable, [3.3FT+6.0FT] PD 60W Fast Charger Nylon Braided Cord for Type C/MicroUSB and i-Product【3 in 1 USB C to USB C cable】- This cable has 3 different magnet connectors, suitable for Type-c, Micro-USB devices and i-Product. (NOTE: This is the USB-C output cable, you may need a USB C wall charger to charge your device.) 【PD 60W Fast Charge】- YKZ type C to type C Cable supports Power Delivery fast charge up to 20V/3A (max), 15V/3A, 12V/3A, 9V/3A, 5V/3A at 60W with USB-C PD adapters such as PD 18W/30W/45W/60W. (Note: Charging speeds also depends on your device and charger. ) 【Magnetic Charging Cable】- Magnetic charging cable has a strong magnet that can attract the phone without falling off. You can easily connect with one hand while driving or doing other things. Besides, Magnetic tips can be used as a du

In [15]:
print(SYSTEM_PROMPT)


I am building a RAG application. I have a collection of 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of the products we have available.
I will provide all of the available products to you with IDs of each chunk.
I want you to come up with 30 questions to which the answers could be grounded in the chunk context.
As an output I need you to provide me the list of questions and the IDs of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Construct 10 that could use multipple chunks in the answer.
Construct 15 questions that could use single chunk in the answer.
Also, include 5 questions that can't be answered with the available chunks.

<OUTPUT JSON SCHEMA>
{
  "type": "array",
  "items": {
    "type": "object",
    "properties": {
      "question": {
        "type": "str

In [16]:
response = openai.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]
)

print(response.choices[0].message.content)

```json
[
  {
    "question": "Do you have any Bluetooth speakers available for outdoor parties?",
    "chunk_ids": ["B0C999S8YM", "B09ZNKRFSC", "B0B93MRTWF"],
    "answer_example": "Yes, we have several Bluetooth speakers suitable for outdoor parties such as the BOGASING Bluetooth Speakers (M4) with IPX7 waterproofing and loud stereo sound, the JBL Flip 6 which is also IPX7 waterproof and has a 12-hour playtime, and the Tronsmart Bang Mini speaker, which offers 50W wireless stereo sound, IPX6 waterproof rating, and a beat-driven light show.",
    "reasoning": "These three chunks describe portable Bluetooth speakers with features such as waterproofing, portability, and loud sound, making them ideal for outdoor parties."
  },
  {
    "question": "Which products support charging iPhones safely with MFi certification?",
    "chunk_ids": ["B0CGHG8Y7H", "B0BDRRTMR7", "B0BM961XHH"],
    "answer_example": "Yes, we have multiple MFi certified products for charging iPhones, including the Coiled

In [ ]:
import json, re, ast

raw_text = response.choices[0].message.content or ""
text = raw_text.strip()

# Remove all leading/trailing code fences if present
text = re.sub(r"^```(?:json)?\s*|\s*```$", "", text, flags=re.S)

# Normalize invisible characters and smart quotes
text = (text
    .replace("\u200b", "")
    .replace("\u201c", '"').replace("\u201d", '"')
    .replace("\u2018", "'").replace("\u2019", "'")
)

# Extract the first balanced top-level JSON array

def extract_first_json_array(s: str) -> str:
    in_string = False
    escape = False
    depth = 0
    start_idx = -1
    for i, ch in enumerate(s):
        if ch == '"' and not escape:
            in_string = not in_string
        if ch == '\\' and not escape:
            escape = True
            continue
        escape = False
        if in_string:
            continue
        if ch == '[':
            if depth == 0:
                start_idx = i
            depth += 1
        elif ch == ']':
            if depth > 0:
                depth -= 1
                if depth == 0 and start_idx != -1:
                    return s[start_idx:i+1]
    return s

text = extract_first_json_array(text)

# Remove trailing commas before } or ] (common LLM artifact)
text = re.sub(r",\s*([}\]])", r"\1", text)

# Try strict JSON first; then fall back
try:
    json_output = json.loads(text)
except Exception:
    try:
        import json5  # optional, only if installed
        json_output = json5.loads(text)
    except Exception:
        # Last-resort: coerce to Python literal
        py_like = (text
            .replace('true', 'True')
            .replace('false', 'False')
            .replace('null', 'None')
        )
        json_output = ast.literal_eval(py_like)

In [25]:
len(json_output)

39

In [26]:
json_output

[{'question': 'Do you have any Bluetooth speakers available for outdoor parties?',
  'chunk_ids': ['B0C999S8YM', 'B09ZNKRFSC', 'B0B93MRTWF'],
  'answer_example': 'Yes, we have several Bluetooth speakers suitable for outdoor parties such as the BOGASING Bluetooth Speakers (M4) with IPX7 waterproofing and loud stereo sound, the JBL Flip 6 which is also IPX7 waterproof and has a 12-hour playtime, and the Tronsmart Bang Mini speaker, which offers 50W wireless stereo sound, IPX6 waterproof rating, and a beat-driven light show.',
  'reasoning': 'These three chunks describe portable Bluetooth speakers with features such as waterproofing, portability, and loud sound, making them ideal for outdoor parties.'},
 {'question': 'Which products support charging iPhones safely with MFi certification?',
  'chunk_ids': ['B0CGHG8Y7H', 'B0BDRRTMR7', 'B0BM961XHH'],
  'answer_example': 'Yes, we have multiple MFi certified products for charging iPhones, including the Coiled Lightning Cable for Carplay (B0CGH

In [ ]:
# since out synthetic data created by llm has hallucinated the chunk id
removed_item = json_output.pop(6)
removed_item

{'question': 'What are my options for expanding USB-C connectivity on a laptop?',
 'chunk_ids': ['B0BCP8SPY5', 'B0CQ39P4M7', 'B09NKXF9YQ'],
 'answer_example': 'You can use the Linkwin 8 in 1 USB C Adapter (B0BCP8SPY5) to add HDMI, Ethernet, USB-A, SD/TF, and charging ports, or the YKZ 3 in 1 Magnetic C to C Cable (B09NKXF9YQ) for versatile charging and data transfer between devices.',
 'reasoning': 'These products allow for expansion or connectivity using USB-C ports (hub and cable with different connectors).'}

In [51]:
points = qdrant_client.scroll(
collection_name="Amazon-items-collection-00",
scroll_filter=Filter(
    must=[
        FieldCondition(
            key="parent_asin",
            match=MatchValue(value="B0BYYLJRHT")
        )
    ]
),
limit=100,
with_payload=True,
with_vectors=False
)[0]

In [52]:
def get_description(parent_asin: str) -> str:

    points = qdrant_client.scroll(
        collection_name="Amazon-items-collection-00",
        scroll_filter=Filter(
            must=[
                FieldCondition(
                    key="parent_asin",
                    match=MatchValue(value=parent_asin)
                )
            ]
        ),
        limit=100,
        with_payload=True,
        with_vectors=False
    )[0]

    return points[0].payload["description"]

## Create eval dataset in langsmith

In [53]:
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])


In [54]:
dataset_name = "rag-evaluation-dataset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Dataset for evaluating RAG pipeline"
)


In [55]:
for item in json_output:
    print(item["chunk_ids"])
    client.create_example(
        dataset_id=dataset.id,
        inputs={"question": item["question"]},
        outputs={
            "ground_truth": item["answer_example"],
            "reference_context_ids": item["chunk_ids"],
            "reference_descriptions": [get_description(id) for id in item["chunk_ids"]]
        }
    )

['B0C999S8YM', 'B09ZNKRFSC', 'B0B93MRTWF']
['B0CGHG8Y7H', 'B0BDRRTMR7', 'B0BM961XHH']
['B0BDRRTMR7', 'B0C4KD57CC', 'B0B6HZVVXC']
['B0BM49LBPZ', 'B0CJV95LVL']
['B0C2C7NMGP', 'B0C48CL3G9']
['B0B9BPRKH5', 'B09YQ6N6HJ']
['B0B3J1NNGL', 'B0BXRMTZCD']
['B0CJV95LVL']
['B0B3GLBW6R']
['B0B51PZC3L', 'B0C52WNWY6', 'B09Y57K53R']
['B0BK8458S2']
['B09XT2BBJF']
['B0BRPPYQVZ', 'B0C5H7VY45']
['B0BDRRTMR7', 'B0BWSBHYV9']
['B0BLY789SC']
['B09N7BHBV1']
['B0B51PZC3L', 'B0C52WNWY6', 'B09Y57K53R', 'B0B3J1NNGL']
['B0BGGG3S1M']
['B0B23RZWTH', 'B0BDF7RPR7']
['B0B2853GJT']
['B0C5H7VY45']
['B0BGH5QMH7']
['B0B7D794HD']
['B0C2C7NMGP']
['B0B51PZC3L']
['B0B3GLBW6R']
['B09WMS7DHQ']
['B0C22QDH2K']
['B0B6D8XB74']
['B09WVR8K3N']
['B0CJV95LVL']
['B0B23RZWTH']
['B0C4KD57CC']
[]
[]
[]
[]
[]
